# Explanations in AI: Methods, Stakeholders and Pitfalls
<h3 align="center">Tabular Data</h3>
<br>

---

This notebook shows how to build a classifier to predict whether an individuals' loan application will be approved or denied. In more detail: The goal is to predict if the applicant will repay the loan, given an instance of a loan application. To answer this, we will train ML models and explore different techniques to explain these models. 

---

__Dataset:__ 
We will use the [South German Credit dataset](https://archive.ics.uci.edu/ml/datasets/South+German+Credit+%28UPDATE%29) which classifies whether credit contracts have been complied with (1: good) or not (0: bad) by using a set of attributes. The attributes consist of a mix of categorical, boolean and numerical values. Missing values are denoted as N/A. 

---

<a name="0">__Contents of Notebook:__</a>

1. <a href="#1">Download & Read the dataset</a>
2. <a href="#2">Data Processing</a>
    * <a href="#21">Exploratory Data Analysis</a>
    * <a href="#22">Train - Validation - Test Datasets</a>
    * <a href="#23">Data processing with Pipeline and ColumnTransformer</a>
3. <a href="#3">Train a Classifier</a>
    * <a href="#31">Sklearn</a>
    * <a href="#32">PyTorch</a>
4. <a href="#4">Test the Classifier</a>
    * <a href="#41">Sklearn</a>
    * <a href="#42">PyTorch</a>
5.  <a href="#5">Explanations</a>
    * <a href="#51">SHAP Values</a>
    * <a href="#52">Counterfactual Explanations</a>
    * <a href="#53">Integrated Gradients</a>
    * <a href="#54">Influential Instances</a>

---
Attribution: Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [[http://archive.ics.uci.edu/ml](http://archive.ics.uci.edu/ml)]. Irvine, CA: University of California, School of Information and Computer Science. Grömping, U. (2019). South German Credit Data: Correcting a Widely Used Data Set. Report 4/2019, Reports in Mathematics, Physics and Chemistry, Department II, Beuth University of Applied Sciences Berlin.

This notebook uses modified code snippets from [Captum](https://captum.ai/tutorials/Titanic_Basic_Interpret).

In [ ]:
# Operational libraries
import os
import sys
from pathlib import Path
import urllib.request
import zipfile

# Jupyter(lab) libraries
if not sys.warnoptions:
    import warnings

    warnings.filterwarnings("ignore")

# Reshaping/basic libraries
import pandas as pd
import numpy as np

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

plt.style.use("seaborn-colorblind")
from utils.gen_utils import extract_mapping
from utils.viz_utils import plot_boundary_with_ce

# Set the style of plotting to white
sns.set_style("white")

# ML libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

# Explainability libraries
import shap
from shap import maskers
import dice_ml
from captum.attr import IntegratedGradients

# Load JS visualization code to notebook
shap.initjs()

# Neural Net libraries
import torch
from torch import nn

## 1. <a name="1">Download & Read the dataset</a>
(<a href="#0">Go to top</a>)

Let's download and unpack the dataset.

In [ ]:
data_dir = Path("SouthGermanCredit")
data_dir.mkdir(exist_ok=True, parents=True)
compressed_file = data_dir / "SouthGermanCredit.zip"
if not compressed_file.is_file():
    urllib.request.urlretrieve(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/00573/SouthGermanCredit.zip",
        compressed_file,
    )
with zipfile.ZipFile(compressed_file, "r") as f:
    f.extractall(data_dir)

In [ ]:
# open data dictionary and extract feature names
with open("codetable.txt", "r") as file:
    data = file.read()

# extract mapping from data dictionary
features_dict, header_dict = extract_mapping(data)

Let's read in the dataset and rename the column headers to English.

In [ ]:
# load the data
df = pd.read_csv(data_dir / "SouthGermanCredit.asc", sep=" ")

# rename features
df.rename(columns=header_dict, inplace=True)

# rename target
df.rename({"credit_risk": "complied_with_credit_contract"}, axis=1, inplace=True)

# create age group variable and drop age feature
df["age_groups"] = df["age"].apply(lambda x: 1 if x >= 25 else 0)
df.drop("age", axis=1, inplace=True)

## 2. <a name="2">Data Processing</a>
(<a href="#0">Go to top</a>)

### 2.1 <a name="21">Exploratory Data Analysis</a>
(<a href="#2">Go to Data Processing</a>)

We look at number of rows, columns, and some simple statistics of the dataset.

In [ ]:
# print the first few rows
df.head(2)

In [ ]:
# check how many rows and columns we have in the data frame
print("The shape of the dataset is:", df.shape)

In [ ]:
# let's see the data types and non-null values for each column
df.info()

#### Data preparation

Based on the feature names, we can see that we are dealing with multimodal data: a mix of categorical, numerical and boolean values. Let's start by creating list for each feature type. If it looks good, we can separate model features from model target to explore them separately.

In [ ]:
# identify the model target
model_target = "complied_with_credit_contract"

# remaining features are numerical
numerical_features = [
    "amount",
    "duration",
]

# split based on categorical nominal and ordinal
categorical_features_nominal = [
    "age_groups",
    "credit_history",
    "foreign_worker",
    "housing",
    "other_debtors",
    "other_installment_plans",
    "people_liable",
    "personal_status_sex",
    "purpose",
    "savings",
    "status",
    "telephone",
]

categorical_features_ordinal = [
    "employment_duration",
    "installment_rate",
    "job",
    "number_credits",
    "present_residence",
    "property",
]

# map numerical values in dataset to underlying categories
for col in categorical_features_nominal:
    if col != "age_groups":
        df[col] = df[col].apply(lambda x: features_dict[col][x])
    else:
        df[col] = df[col].apply(lambda x: str(x))

Let's set up some additional lists that can be reused later for visualizations and model training.

In [ ]:
# group the categorical features for further analysis
categorical_features = categorical_features_nominal + categorical_features_ordinal

# assign model features
model_features = categorical_features + numerical_features

# print the features and target
print("Model features: ", model_features)
print("Model target: ", model_target)

In [ ]:
# Double check that that target is not accidentally part of the features
model_target in model_features

All good here. We made sure that the target is not in the feature list. If we find the above statement showing `True` we need to remove the target by calling `model_features.remove(model_target)`.

#### Target distribution

Let's check our target distribution. This is the most important column to analyze. We need to check the distribution of the target as well as any skew towards different subpopulations. Eventually we want to compare quality of explanations for two subgroups in the dataset (age $\geq$ 25 and age <25 years old).

In [ ]:
sns.set_style("whitegrid")

# plot target again but split by age group
g = sns.catplot(
    x=model_target,
    kind="count",
    hue="age_groups",
    data=df,
    palette="husl",
    legend=False,
)

g.fig.set_size_inches(8, 4)
plt.title("Histogram of outcomes (split by age group)")
plt.legend(title="age_groups", loc="upper left", labels=["<25y", ">=25y"])
plt.show()

We notice that we are dealing with an imbalanced dataset; there are more examples for one class 1. Interestingly, the average default rate on loans in Germany at the time the dataset was collected was approximately 5 percent. As the target distribution shows a much larger percentage of individuals that did not comply with the credit contract, we can assume that intentional oversampling of the negative class was conducted to better identify individuals who are likely to default. 

In addition, as we split the plot based on age groups we can also observe the following:
- There is only a small fraction of applications from individuals <25y old.
- For the age group <25y the outcome is split at almost 50/50; this could indicate that there is bias present in the data or point at sampling issues.

### 2.2 <a name="22">Train - Validation - Test Datasets</a>
(<a href="#2">Go to Data Processing</a>)

Following general best practice for building Machine Learning models, we want to split our data into train, test and validation set. To obtain these splits, we will use sklearn's [train_test_split()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function.

In [ ]:
train_data, test_data = train_test_split(
    df, test_size=0.1, shuffle=True, random_state=23
)
train_data, val_data = train_test_split(
    train_data, test_size=0.15, shuffle=True, random_state=23
)

# Print the shapes of the Train - Test Datasets
print(
    "Train - Test - Validation datasets shapes: ",
    train_data.shape,
    test_data.shape,
    val_data.shape,
)

### 2.3 <a name="23">Data processing with Pipeline and ColumnTransformer</a>
(<a href="#2">Go to Data Processing</a>)

Let's build a full model pipeline. We need pre-processing split per data type, and then combine everything back into a composite pipeline so that we can transform all features once and for all and then build different models on top. To achieve the desired data prep, we will use sklearns `Pipeline` and `ColumnTransformer`.

In [ ]:
# Preprocess the numerical features
numerical_processor = Pipeline(
    [
        ("num_imputer", SimpleImputer(strategy="mean", add_indicator=True)),
        ("num_scaler", MinMaxScaler()),
    ]
)
# Preprocess the categorical features
categorical_processor_nominal = Pipeline(
    [
        ("cat_imputer", SimpleImputer(strategy="constant")),
        ("cat_encoder_nom", OneHotEncoder(handle_unknown="ignore", drop="if_binary")),
    ]
)
categorical_processor_ordinal = Pipeline(
    [
        ("cat_imputer", SimpleImputer(strategy="constant")),
        (
            "cat_encoder_ord",
            OrdinalEncoder(handle_unknown="error"),
        ),
    ]
)

# Combine all data pre-processors from above
data_processor = ColumnTransformer(
    [
        ("numerical_processing", numerical_processor, numerical_features),
        (
            "categorical_processing_nom",
            categorical_processor_nominal,
            categorical_features_nominal,
        ),
        (
            "categorical_processing_ord",
            categorical_processor_ordinal,
            categorical_features_ordinal,
        ),
    ]
)

# Pipeline with desired data transformers, along with an estimator at the end
pipeline = Pipeline(
    [
        ("data_processing", data_processor),
        ("rf", RandomForestClassifier(max_depth=10, random_state=1)),
    ]
)

# Visualize the pipeline
from sklearn import set_config

set_config(display="diagram")
pipeline

In [ ]:
# Get train data to train the classifier
X_train = train_data.drop(model_target, axis=1)
y_train = train_data[model_target]

# Get validation data to validate the classifier
X_test = test_data.drop(model_target, axis=1)
y_test = test_data[model_target]

# Get train data to train the classifier
X_val = val_data.drop(model_target, axis=1)
y_val = val_data[model_target]

## 3. <a name="3">Train a Classifier</a>
(<a href="#0">Go to top</a>)

We use the pipeline with the desired data transformers, along with a Random Forest estimator for training.


### <a name="31">3.1. Model Training - Sklearn</a>

We train the RandomForest classifier with __.fit()__ on our training dataset. 

In [ ]:
# Fit the classifier to the train data
clf = pipeline.fit(X_train, y_train)

In [ ]:
# Let's predict for the validation dataset
y_val_pred = clf.predict(X_val)

print("Model performance on the validation set:")
print("Validation accuracy:", accuracy_score(y_val, y_val_pred))

We are concerned with model explainability in this notebook, so we will not perform any model tuning at this stage.

### <a name="32">3.2. Model Training - PyTorch</a>

As we want to compare explainability methods that rely on gradients and linear combinations, we will build a different type of model; a simple neural network (NN). For this, we need to cast our data as tensors first. Then we can set up the NN and train it.

In [ ]:
# learn the encoding/transformation
data_processor.fit(X_train)

# convert to tensor - use fit transformer to encode features before converting to tensor
X_train_tensor = torch.tensor(data_processor.transform(X_train)).float()
y_train_tensor = torch.tensor(y_train.values).view(-1, 1).float()

# repeat for validation set
X_val_tensor = torch.tensor(data_processor.transform(X_val)).float()
y_val_tensor = torch.tensor(y_val.values).view(-1, 1).float()

# repeat for test set
X_test_tensor = torch.tensor(data_processor.transform(X_test)).float()
y_test_tensor = torch.tensor(y_test.values).view(-1, 1).float()

# store tensor in TensorDataset and create DataLoader for iteration
dataset = torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)
train_iter = torch.utils.data.DataLoader(dataset, batch_size=20, shuffle=True)

In [ ]:
# initialize NN hyperparameters
num_epochs = 60
learning_rate = 0.001

# define architecture of NN
size_hidden1 = 24
size_hidden2 = 8

torch.manual_seed(1)  # set seed for reproducibility.


class LoanModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(49, size_hidden1)
        self.activation_1 = nn.ReLU()
        self.lin2 = nn.Linear(size_hidden1, size_hidden2)
        self.activation_2 = nn.Sigmoid()
        self.lin3 = nn.Linear(size_hidden2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input):
        return self.sigmoid(
            self.lin3(self.activation_2(self.lin2(self.activation_1(self.lin1(input)))))
        )

    def train(net_model, num_epochs=num_epochs, verbose=True):
        optimizer = torch.optim.Adam(net_model.parameters(), lr=learning_rate)
        # define loss function
        criterion = nn.BCELoss(reduction="none")
        # initialize empty lists to later track validation and training loss across epochs
        val_losses = []
        trn_losses = []
        for epoch in range(1, num_epochs):
            training_loss = 0.0
            for inputs, labels in train_iter:
                # zero parameter grads
                optimizer.zero_grad()
                # forward pass
                outputs = net_model(inputs)
                # calculate loss
                loss = criterion(outputs, labels).sum()
                # accumulating running loss
                training_loss += loss.item()
                # compute grads
                loss.backward()
                # updated weights based on computed gradients
                optimizer.step()

            # Get validation predictions
            val_predictions = net_model(X_val_tensor)

            # Calculate the validation loss
            validation_loss = criterion(val_predictions, y_val_tensor).sum().item()

            # Take the average losses
            training_loss /= len(y_train_tensor)
            validation_loss /= len(y_val_tensor)

            # Append to list
            trn_losses.append(training_loss)
            val_losses.append(validation_loss)

            if verbose == True:
                # Print the losses every 15 epochs
                if (epoch == 0) or ((epoch + 1) % 15 == 0):
                    print(
                        "Epoch %s. Train_loss %f Validation_loss %f "
                        % (epoch, training_loss, validation_loss)
                    )
        if verbose == True:
            plt.plot(trn_losses, label="Training Loss")
            plt.plot(val_losses, label="Validation Loss")
            plt.title("Loss values")
            plt.xlabel("Epoch")
            plt.ylabel("Loss")
            plt.legend()
            plt.show()


net = LoanModel()
net.train()

We can see that after 15 epochs the model starts to overfit on the training data. Let's retrain the model with only  15 epochs.

In [ ]:
net = LoanModel()
net.train(num_epochs=15, verbose=False)

In [ ]:
# pass validation dataset through NN for predictions
outputs = net(X_val_tensor)

print("Model performance on the validation set:")
print(
    "Validation accuracy:",
    np.round(
        accuracy_score(
            # use torch round to convert probability values to classes (default round threshold is 0.5)
            y_val_tensor.detach().numpy(),
            torch.round(outputs).detach().numpy(),
        ),
        2,
    ),
)

## 4. <a name="4">Test the Classifier</a>
(<a href="#0">Go to top</a>)

Let's now evaluate the performance of the trained classifiers on the test dataset. We don't expect the performance to be different on the test dataset as there was no tuning conducted, hence the distribution of the test and validation data should be very similar.

### <a name="41">4.1. Testing the Classifier - Sklearn</a>

In [ ]:
# use the fitted model to make predictions on the test dataset
test_predictions = clf.predict(X_test)

print("Model performance on the test set:")
print("Test accuracy:", accuracy_score(y_test, test_predictions))

plt.style.use("seaborn-white")
fig, ax = plt.subplots(figsize=(4, 4))
ConfusionMatrixDisplay.from_estimator(clf, X_test, y_test, ax=ax)
plt.show()

### <a name="42">4.2. Testing the Classifier - PyTorch</a>

In [ ]:
# pass test dataset through NN for predictions
outputs = net(X_test_tensor)

print("Model performance on the test set:")
print(
    "Test accuracy:",
    accuracy_score(
        y_test_tensor.detach().numpy(), torch.round(outputs).detach().numpy()
    ),
)

plt.style.use("seaborn-white")
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(4, 4))
cmd = ConfusionMatrixDisplay(
    confusion_matrix(
        y_test_tensor.detach().numpy(), torch.round(outputs).detach().numpy()
    )
)
cmd.plot(ax=axs)
plt.show()

Now that we have predictions, we can look at some explanations for those predictions. Most explainers work with Sklearn or deep learning frameworks, such as PyTorch, as backend.

## 5. <a name="5">Explanations</a>
(<a href="#0">Go to top</a>)

### <a name="51">5.1. SHAP</a>
(<a href="#5">Go to Explanations</a>)

SHAP is an attributive explanation method; it quantifies the impact (called attribution) of each feature on the model prediction. SHAP values provide an answer to the question: "What are the contributions of the input features toward the output prediction?". SHAP (SHapley Additive exPlanations) is a game theoretic approach to explain the output of any machine learning model. It connects optimal credit allocation with local explanations using the classic Shapley values from game theory (see [paper](https://proceedings.neurips.cc/paper/2017/hash/8a20a8621978632d76c43dfd28b67767-Abstract.html) for details).

SHAP iteratively “turns off” features and notes the marginal effect on the prediction. This is done for all possible permutations of features to compute a weighted average of the marginal effects. Because all permutations need to be considered SHAP has high computational complexity (some approximations/simplifications are implemented in the library to speed up the process). Let's have a look at SHAP values and plots for our dataset. To make the explanations more readable, we extract the features names (for encoded categorical columns) and create a small sample data frame.

In [ ]:
# extract feature names (this is required because onehot encoding created new columns)
ft_names = (
    numerical_features
    + list(
        data_processor.transformers_[1][1]
        .named_steps["cat_encoder_nom"]
        .get_feature_names_out(categorical_features_nominal)
    )
    + list(
        data_processor.transformers_[2][1]
        .named_steps["cat_encoder_ord"]
        .get_feature_names_out(categorical_features_ordinal)
    )
)

In [ ]:
# create sample from X_test with the feature names attached
X_sample = pd.DataFrame(data_processor.transform(X_test), columns=ft_names).copy(
    deep=True
)

# needs to be 0 or 1 for binary classification; 1: credit complied, 0: not complied (risky customer)
low_risk = 1
high_risk = 0

Now that we have everything prepared, let's start using the SHAP explainer to get the SHAP values. Generally it is advisable to use the training dataset for the feature perturbation; to reduce computational run-time, we will proceed with the small sample dataset that we created in the cell above.

In [ ]:
# create the explanations by passing in the model and the data, we can also use feature_names=ft_names
explainer_shap = shap.Explainer(
    clf[-1], feature_perturbation="tree_path_dependent", seed=1
)

# extract SHAP values
shap_values = explainer_shap(X_sample)

# before proceeding further, we want to scale the numerical features back to the original values - careful as this will update the values in X_samples too
shap_values.data[:, : len(numerical_features)] = (
    clf[0]
    .named_transformers_["numerical_processing"]
    .inverse_transform(shap_values.data[:, : len(numerical_features)])
)

#### <a name="511">5.1.1. Global SHAP explanations</a>
(<a href="#51">Go to SHAP</a>)

Let's start with a global look at the model behavior first. The plot below sorts features by the sum of SHAP value magnitudes over all samples, and uses SHAP values to show the distribution of the impacts each feature has on the model output. The color represents the feature value (red high, blue low).

In [ ]:
# create beeswarm plot for global explanations
shap.plots.beeswarm(shap_values[:, :, low_risk])

In [ ]:
# create beeswarm plot for global explanations
shap.plots.beeswarm(shap_values[:, :, low_risk])

There is a lot going on in this plot above, so let's look at a simpler visualization that uses the mean absolute value of the SHAP values for each feature to summarize importance:

In [ ]:
# plot that takes absolute SHAP values and averages across all data points
shap.plots.bar(shap_values[0:, :, low_risk])

<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
<i class="fa fa-exclamation-circle" style="color:red"></i> Careful with this plot, it uses the absolute attribution values, so any directional information is lost! The absolute values are necessary to avoid cancelling of the positive/negative effects.

#### <a name="512">5.1.2. Local SHAP explanations</a>
(<a href="#51">Go to SHAP</a>)

We will now look at a local explanation created with SHAP. The explanation starts with a base value and will add & subtract SHAP values that push the model output from the base value (the average model output over the training dataset we passed) to the model output. Features pushing the prediction higher are shown in red, those pushing the prediction lower are in blue.

In [ ]:
# select a data point to analyze
sample_to_explain = 34  # needs to be between 0 and length of data frame

In [ ]:
# Let's first look at the predicted class for the data point we are interested in
clf.predict_proba(X_test)[sample_to_explain : sample_to_explain + 1], clf.predict(
    X_test
)[sample_to_explain : sample_to_explain + 1]

In [ ]:
# create waterfall plot for being a credit risk for the data point in question (this individual has a low chance of not being a risk)
shap.plots.waterfall(shap_values[sample_to_explain][:, high_risk])

The expected value to have high credit risk in this dataset is approximately 0.3; for this particular instance the probability to have high risk is more than double that. We can use the visualization of the SHAP values above to better understand why this customer has such high risk. For example, we can see that this customer requested a very high contract duration, does not have any savings, has no checking account and also no significant property in their name (`property = 3` corresponds to "car or other", whereas `1` and `2` indicate the customer has real estate or significant savings in their name).

##### EXERCISE
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/fontawesome.min.css">
<i class="fa-solid fa-book-open"></i> Create a SHAP waterfall plot for the data point that is most certainly not a credit risk (you can rank by outcome probability). Write a short explanation for a hypothetical customer after looking at the waterfall plot.

In [ ]:
##########################################################################################
# Complete the code for the exercise below:


##########################################################################################

When computing attributions for tree structured models such as random forests, the SHAP library uses an algorithm that estimates the Shapley values based on the tree model structure itself; to get a fast approximation of the expected model output conditional to the "in-coalition" features SHAP uses the leaf membership of training examples. In fact, there are other ways to compute feature attributions with SHAP too (beyond the tree-path dependent feature perturbation method).

<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/fontawesome.min.css">
<i class="fa-solid fa-circle-exclamation" style="color:red"></i> Depending on how the feature attribution is calculated, you will obtain different SHAP values. The source of these differences stems from the way we treat "out-of-coalition" features. Let's see a few of these in the examples below. 

To try different explainers, we create a utility class, `AttributiveExplanation`.

___
For more details, have a look at "[The many Shapley values for model explanation](https://arxiv.org/abs/1908.08474)" (Sundararajan et al., 2019).

In [ ]:
# define a utility class for storing attributions
class AttributiveExplanation:
    def __init__(self, shap_explainer, data):
        self.explainer = shap_explainer
        self.data = data
        self.values = None

    def compute(self):
        self.values = self.explainer(self.data)
        return self

    def plot(self, data_point=sample_to_explain, show=True):
        shap.plots.waterfall(self.values[data_point][:, high_risk], show=True)

    def sv(self, data_point=sample_to_explain):
        return self.values.values[data_point][:, high_risk]

The tree-path dependent approach is “true to the data” because it avoids basing the computation of SHAP values on unrealistic data instances. It achieves this by constraining the sampling of “unknown” features (those not belonging to the coalition under study) to a range of values (i.e. partition of the feature space) allowed by the decision tree, effectively conditioning on the features that split prior nodes. With the `tree-path-dependent` setting, there is no background data required as the background distribution can be inferred from the structure of the model itself; this is useful when trying to understand a model without access to the training data.

In [ ]:
# let's plot again the local attributions using the same explainer as before (clf[-1] is used to access the model)
explainer_base = shap.Explainer(
    clf[-1], feature_perturbation="tree_path_dependent", seed=1
)

# compute SHAP values
attributions_tree_path = AttributiveExplanation(explainer_base, X_sample).compute()

# show output as plot
attributions_tree_path.plot(data_point=17)

In [ ]:
# now we look at another usage of the tree explainer that instead breaks the dependencies between features according to the rules dictated by causal inference - "interventional"
explainer_interventional = shap.TreeExplainer(
    clf[-1],
    feature_perturbation="interventional",
    data=X_sample,
    model_output="probability",
)

# compute SHAP values
attributions_interventional = AttributiveExplanation(
    explainer_interventional, X_sample
).compute()

# show output as plot
attributions_interventional.plot(data_point=17)

We can see that the explanations differ based on the perturbation method that is being used to generate the outputs. Let's have 

##### EXERCISE
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/fontawesome.min.css">
<i class="fa-solid fa-book-open"></i> Create another SHAP explanation by using a fixed value for the hidden features. To achieve this, you have to specify the <code>masker</code> parameter in the explainer which expects an array of values or a function. 

As simple first approach, you could calculate the mean across the dataset and pass that as array to the masker. Careful if you plan to use the <code>AttributiveExplanation</code> class; make sure to match the expected input format.

In [ ]:
##########################################################################################
# Complete the code for the exercise below:


##########################################################################################

We can notice that different ways of treating out-of-coalition features results in largely different attributions. This is because the underlying _game_ on which you compute the Shapley value changes, and so does the interpretation of the explanations! Let's have a look at other limitations next.

<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
<i class="fa fa-exclamation-circle" style="color:red"></i> SHAP (and other attribution methods) are also very sensitive to perturbations to the input. For instance, a small perturbation that does not affect the prediction may still alter the attribution value. Small changes in the data should not affect the predictions of the model or the explanation we get, yet the following example shows that small perturbations can indeed change the SHAP explanation (we will see that this happens for other explainability methods too). 

___
More details about this can be found in the paper "[On the Robustness of Interpretability Methods](https://arxiv.org/abs/1806.08049)".

In [ ]:
# create sample from X_test with the feature names attached
X_sample_per = X_sample.copy(deep=True)

# define random noise
mu, sigma = 0.05, 0.15

# creating a noise with the same dimension as the dataset
np.random.seed(seed=4)
noise = np.random.normal(mu, sigma, [X_sample_per.shape[0], len(numerical_features)])

# add noise to data
X_sample_per.values[:, : len(numerical_features)] = (
    X_sample_per.values[:, : len(numerical_features)] + noise
)

# Let's first look at the predicted class for the data point we are interested in
clf[-1].predict_proba(X_sample_per)[sample_to_explain : sample_to_explain + 1], clf[
    -1
].predict(X_sample_per)[sample_to_explain : sample_to_explain + 1]

These results match the output that we received before adding a tiny amount of random noise really well (as reminder, we previously obtained the following class probabilities: `array([[0.72354782, 0.27645218]])`). Clearly the model itself is not heavily influenced by the noise. Let's see what the perturbation did to the SHAP values.

In [ ]:
# let's plot again the local attributions using the same explainer as before
explainer_perturbed = shap.Explainer(
    clf[-1], feature_perturbation="tree_path_dependent", seed=1
)

# compute SHAP values
attributions_perturbed = AttributiveExplanation(
    explainer_perturbed, X_sample_per
).compute()

# show output as plot
ax1 = plt.subplot(211)
attributions_perturbed.plot(show=True)

# add margins
ax1.margins(0.5)

# show plot of undisturbed data for comparison
ax2 = plt.subplot(212)
shap.plots.waterfall(shap_values[sample_to_explain][:, high_risk], show=True)

We can clearly see that the small change in `amount` has resulted in change in the Shapley values. Let's have a look at one final limitation of SHAP values next.

<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
<i class="fa fa-exclamation-circle" style="color:red"></i> The Shapley value is a way to distribute the contributions of each single features toward achieving a target prediction. As such it is a descriptive tool and can lack actionability. We shouldn't expect that changing the instance features using (any instantiations of) the Shapley value would yield to any predictable result. This is, e.g. in contrast with the gradient, which indicates the direction of local maximum increase of a function.

Let's look at a toy example to see this clearly; this is inspired by [Problems with Shapley-value-based explanations as feature importance measures](https://arxiv.org/abs/2002.11097).


In [ ]:
# a simple quadratic function with maximum in x = 1
def f(x):
    return 2 - (x - 1) ** 2


# its gradient
def grad(x):
    return 2 - 2 * x


# let's consider the scalar case for ease of visualization
def shap_values_f(x, baseline=0.0):
    return shap.Explainer(f, np.array([[baseline]]))(x).values

In [ ]:
# create sample datapoints
xs = np.linspace(-1, 3)

# compute SHAP values
sh = shap_values_f(xs[:, None]).reshape(-1)

In [ ]:
# plot function, gradient and Shapley values
plt.plot(xs, f(xs), label="f(x) = 2 - (x - 1)^2")
plt.plot(xs, grad(xs), label="Gradient of f")
plt.plot(xs, sh, ".", label="Shapley value with baseline = 0")
axes1 = (-3, 4), (0, 0)
axes2 = (0, 0), (-3, 3)

# plot intersecting lines
plt.plot(*axes1, color="k")
plt.plot(*axes2, color="k")

# add legend and show plot
plt.legend(loc=0)
plt.xlim((-1, 3))
plt.ylim((-2.5, 2.5))
plt.show()

At $x=1$ the Shapley value is 1. Since this is a positive number, one may think that by increasing the (single) feature then also the output should increase, but this is clearly not true. The gradient instead carries  this information. On the other hand, the Shapley value tells how we reach the model output starting from the baseline output (which in this case is $f(0) = 1$.

Increasing the feature value does not automatically increase the Shapley value. Because of this, we can say that SHAP lacks actionability which is one of the main desiderata of explainer methods.

#### <a name="513">5.1.3. Summary SHAP</a>
(<a href="#51">Go to SHAP</a>)

Let's summarize this section:
- We reviewed different choices of feature perturbation methods and checked Shapley values for robustness
- We saw how SHAP values are not actionable as increase/decrease in feature values does not result in changes in the Shapley values.

In the next section, we will review counterfactuals which are easy to comprehend, and can be used to offer a path of recourse to customers.

### <a name="52">5.2. DiCE Counterfactuals</a>
(<a href="#5">Go to Explanations</a>)


**Counterfactual Explanations (CEs)** are imaginary versions of model inputs that receive a different prediction than the original input. To be actionable, counterfactuals should be as similar as possible to the original input in terms of feature values. To find a counterfactual, the difference, $d$,  between the original input, $x$, and the imaginary input, $x_c$, are calculated. Obviously there are many possible imaginary inputs, so the goal is to find the one example, $x_c$, that is closest to the original in terms of feature values, yet has a different outcome:

> argmin $d(x, x_C) \; s.t. \; f(x_c) \neq y$

To be practical, the feature values should be feasible and ideally, a counterfactual should change as few features as possible. 

Below, we will take an input that was classified by our models as credit risk. We will generate CEs -- counterfactual versions of this input -- showing the changes required so that the classifier prediction changes to no risk. Generating counterfactual explanations in code is a simple three-step process: initialize dataset, point to model and then invoke DiCE to generate counterfactual examples for any input.

In [ ]:
# Dataset for training an ML model
d = dice_ml.Data(
    dataframe=train_data,
    continuous_features=numerical_features + categorical_features_ordinal,
    outcome_name=model_target,
    type_and_precision="int",
)

# Using sklearn backend
m = dice_ml.Model(model=clf, backend="sklearn")

# Using random method for generating CFs
exp = dice_ml.Dice(d, m, method="random")

#### <a name="521">5.2.1. Local counterfactual explanations</a>
(<a href="#52">Go to CE</a>)

DiCE implements counterfactual (CF) explanations that provide this information by showing feature-perturbed versions of the same person who would have received the loan, e.g., you would have received the loan if your income was higher by 10,000 USD. In other words, it provides "what-if" explanations for model output and can be a useful complement to other explanation methods, both for end-users and model developers.

In [ ]:
e1 = exp.generate_counterfactuals(
    X_test[sample_to_explain : sample_to_explain + 1],
    total_CFs=4,
    desired_class=1,
    random_seed=4,
)
e1.visualize_as_dataframe(show_only_changes=True)

<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
<i class="fa fa-exclamation-circle" style="color:red"></i> Not all counterfactual explanations may be feasible for a customer. In general, counterfactuals closer to an individuals' profile will be more feasible. Diversity is also important to help an individual choose between multiple possible options. DiCE provides tunable parameters for diversity and proximity to generate different kinds of explanations.

In [ ]:
e2 = exp.generate_counterfactuals(
    X_test[sample_to_explain : sample_to_explain + 1],
    total_CFs=4,
    desired_class="opposite",
    proximity_weight=1.5,
    diversity_weight=2.0,
    random_seed=2,
)

e2.visualize_as_dataframe(show_only_changes=True)

<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
<i class="fa fa-exclamation-circle" style="color:red"></i> Some features such as one's age or gender, are impossible to change for a credit loan applicant. Therefore, DiCE also allows inputting a list of features to vary.

DiCE also supports simple constraints on features that reflect practical constraints (e.g., housing is 'rent' using the `permitted_range` parameter). Banks could not expect someone to buy a house to change their housing status from 'rent' to 'own' so it makes sense to restrict the permitted range. 

In [ ]:
vary_features = ["amount", "duration"]

# Restricting housing to be either {'rent'}
e3 = exp.generate_counterfactuals(
    X_test[sample_to_explain : sample_to_explain + 1],
    desired_class="opposite",
    total_CFs=4,
    features_to_vary=vary_features,
    permitted_range={"amount": [200, 6000]},
    random_seed=5,
)

e3.visualize_as_dataframe(show_only_changes=True)

We see a shorter credit duration as counterfactual, so this is something that the applicant could indeed aspire to pursue. We can also observe that the permitted range was considered as defined (there are no counterfactuals with amounts higher than the specified 6000).

##### EXERCISE

<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/fontawesome.min.css">
<i class="fa-solid fa-book-open"></i> Create three CF explanations that restricts the permitted <code>duration</code> to be between 10 and 60 (inclusive) and <code>employment_duration</code> to values greater than 2.

In [ ]:
##########################################################################################
# Complete the code for the exercise below:


##########################################################################################

While setting permitted ranges manually can be helpful, it can also be very tedious to do manually. Rather than testing all possible value combinations, we can simply plot the boundary of the counterfactuals:

In [ ]:
query_instance = X_test[sample_to_explain : sample_to_explain + 1]
query_ce_df = e3.cf_examples_list[0].final_cfs_df

plot_boundary_with_ce(
    query_instance,
    query_ce_df,
    clf,
    X_train,
    vary_features,
    categorical_features,
    color_mapping={0: "Red", 1: "Green"},
)

##### EXERCISE

<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/fontawesome.min.css">
<i class="fa-solid fa-book-open"></i> Recreate the plot above but this time use 'job' and 'duration'. What is the maximum accepted duration that will still give a positive outcome for the customer?

In [ ]:
##########################################################################################
# Complete the code for the exercise below:


##########################################################################################

It is also possible to create counterfactuals when the model is a deep neural network; the below code examples shows an implementation. More details about the method to find explanations can be found in "[Explaining Machine Learning Classifiers through Diverse Counterfactual Explanations](https://arxiv.org/abs/1905.07697)".

In [ ]:
# initialize dataset; the PyTorch model in this notebok expects the transformed dataset as input, so we need to recreate this here
d_pyt = dice_ml.Data(
    dataframe=pd.concat(
        [
            pd.DataFrame(data_processor.transform(X_test), columns=ft_names),
            pd.DataFrame(y_test.values, columns=[model_target]),
        ],
        1,
    ),
    continuous_features=numerical_features + categorical_features_ordinal,
    outcome_name=model_target,
    type_and_precision="int",
)

# get sample datapoint transformed
x_transformed = pd.DataFrame(
    data_processor.transform(X_test[sample_to_explain : sample_to_explain + 1]),
    columns=ft_names,
)

# use PyTorch backend
m_pyt = dice_ml.Model(model=net, backend="PYT")

# initialize explainer
exp_pyt = dice_ml.Dice(d_pyt, m_pyt, method="gradient")

# pass in instance and create counterfactuals
e_pyt = exp_pyt.generate_counterfactuals(
    query_instances=x_transformed,
    desired_class="opposite",
    total_CFs=4,
)

In [ ]:
# inverse transformation for numerical features
x_num = data_processor.named_transformers_["numerical_processing"].inverse_transform(
    x_transformed[numerical_features]
)

# update values in df
x_transformed.loc[:, numerical_features] = x_num.round(-1)
x_transformed.loc[:, model_target] = y_test[
    sample_to_explain : sample_to_explain + 1
].values

# show df
x_transformed

In [ ]:
# extract counterfactuals to dataframe
e_pyt_df = e_pyt.cf_examples_list[0].final_cfs_df.copy(deep=True)

# inverse transformation numerical features
e_pyt_num_fts = data_processor.named_transformers_[
    "numerical_processing"
].inverse_transform(e_pyt_df[numerical_features])

# update values in counterfactal df
e_pyt_df.loc[:, numerical_features] = e_pyt_num_fts.round(-1)

# show counterfactuals
e_pyt_df

Now that we've seen different ways to create local explanations using counterfactuals, we can move on to the next section.

#### <a name="522">5.2.2. Global counterfactual explanations</a>
(<a href="#52">Go to CE</a>)

It is also possible to look at global feature importance with counterfactuals. Intuitively, a feature that is changed more often to generate a counterfactual is an important feature. Hence, it is possible to use summaries of counterfactual examples to estimate importance of features.

In [ ]:
global_ce = exp.global_feature_importance(
    X_test[0:10], total_CFs=10, posthoc_sparsity_param=None
)
print(global_ce.summary_importance)

#### <a name="523">5.2.3. Summary CE</a>
(<a href="#52">Go to CE</a>)

Let's summarise some of the challenges we encountered when working with counterfactuals.

<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
<i class="fa fa-exclamation-circle" style="color:red"></i> The first first challenge is how to define "closeness" between the original data point and the counterfactual. It turns out that this is actually not trivial as features can vary on different scales, and loss can therefore vary non-linearly with the feature value. For instance, `amount` in our dataset can vary in the thousands but `duration` only varies in the tens. When working with counterfactuals it can be helpful to consult a domain expert to provide a domain-specific distance functions.
<br>
<br>
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
<i class="fa fa-exclamation-circle" style="color:red"></i> Furthermore, to obtain counterfactuals an optimization problem has to solved which can be computationally challenging, especially when the dataset contains categorical features which turn the problem into a combinatorial optimization problem. 
<br>
<br>
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
<i class="fa fa-exclamation-circle" style="color:red"></i> Finally, features highlighted in counterfactuals may not always have a large attribution. This can happen when the majority of the datapoints in the dataset has the same (or similar) feature value. For example, in our dataset the `number_credits` feature is the same for almost all loan applicants. This will lead to attribution methods such as SHAP to not attribute much value to it. However, decreasing the number of credits (paying off other debtors) may very well be a valid recourse that would make the applicant credit worthy.

<br>
<br>

Let's summarize this section:
- We implemented counterfactuals for different model backends (sklearn & PyTorch) and visualized the results
- We reviewed different settings for creating counterfactuals that consider proximity and feasibility of the resulting counterfactuals

In the next section, we will review integrated gradients as a method that works very well for various deep learning models and is a computationally very efficient method to create explanations (in particular for non-tabular data).


### <a name="53">5.3. Integrated Gradients (IG)</a>
(<a href="#5">Go to Explanations</a>)

Gradients are useful way to measure the importance of features in parametric models (specifically, the model has to be differentiable to be able to obtain gradients in the first place). Gradients describe the local slope of an arbitrary function, $f(x)$ (here: the model), and allow us to quantify whether taking a small step in either direction on the function has a small or large effect on the result. In the context of explainability, we want to understand if a change in the input feature has an impact on the output (prediction), so we calculate the gradient of the input with respect to the output. 


<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
<i class="fa fa-exclamation-circle" style="color:red"></i> However, simple gradient based explanations can be noisy and suffer from a saturation problem: whenever the model is saturated, the gradient will be 0 (see "<a href="https://arxiv.org/abs/1704.02685">Learning Important Features Through Propagating Activation Differences</a>" for details).



#### <a name="531">5.3.1. IG method</a>
(<a href="#53">Go to IG</a>)

Integrated Gradients (IG) is a technique that bypasses the saturation problem by attributing a differentiable model's prediction to its base features. IG creates a straight line path in the feature space, from a given input data point to a certain baseline input (e.g., all-zero baseline), and integrates the gradient of the output (prediction) with respect to input features along this path.

Gradient-based explanation methods try to explain a given prediction by using the gradient of (i.e. change in) the output with respect to the input features. Computing (integrated) gradients is well supported in most machine learning frameworks which makes gradient-based explanations a sensible choice when working with neural networks in particular. IG will also work for different data modalities (e.g., image data - see `Explainability_ImageData.ipynb`) and is computationally efficient compared to other methods as there are no permutations or large amounts of repeat calculations required.

In [ ]:
# instantiate the IG method
ig = IntegratedGradients(net)

In [ ]:
# Helper method to print importances and visualize distribution
def visualize_importances(
    feature_names,
    importances,
    verbose=False,
    title="Average Feature Importances",
    plot=True,
    axis_title="Features",
):
    if verbose == True:
        print(title)
        for i in range(len(feature_names)):
            print(feature_names[i], ": ", "%.3f" % (importances[i]))
    x_pos = np.arange(len(feature_names))
    if plot:
        plt.figure(figsize=(20, 6))
        plt.bar(x_pos, importances, align="center")
        plt.xticks(x_pos, feature_names, wrap=False, rotation=90)
        plt.xlabel(axis_title)
        plt.title(title)

IG expects a baseline; when baseline is not provided, Captum uses a zero scalar corresponding to each input tensor. For a network with multiple possible outputs, a target index must also be provided, defining the index of the output for which gradients are computed. For this example, we don't need to provide a target as there is no target index necessary when the model provides a scalar value per example. 

The returned values of the attribute method are the attributions, which match the size of the given inputs, and delta, which approximates the error between the approximated integral and true integral.

In [ ]:
ig_attr_train = ig.attribute(X_train_tensor.requires_grad_(), n_steps=50)
visualize_importances(ft_names, np.mean(ig_attr_train.detach().numpy(), axis=0), False)

We notice that `installment_rate` and `employment_duration` have the highest negative and positive average feature importance. Interestingly, `employment_duration` was not a feature that was considered important by other explainability methods so far. Employment duration is positively correlated with the output, whereas installment_rate is negatively correlated.

##### EXERCISE

<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/fontawesome.min.css">
<i class="fa-solid fa-book-open"></i> Recreate the plot above but this time use <code>InputXGradient</code>. 

Make sure to import the explainer first with `from captum.attr import InputXGradient`. More details about the method can be found [here](https://captum.ai/api/input_x_gradient.html). How do the results differ?

In [ ]:
##########################################################################################
# Complete the code for the exercise below:


##########################################################################################

#### <a name="532">5.3.2. IG method</a>
(<a href="#53">Go to IG</a>)

Let's summarize this section:
- We discussed the difference between 'vanilla' gradient-based methods and IG
- We implemented IG for the custom NN and created a visualization to show the average feature importances

In the next section, we will review one final method that tries to extract explanations from the dataset that was used to train the model itself by finding influential training instances.

### <a name="54">5.4. Deletion Diagnostic</a>
(<a href="#5">Go to Explanations</a>)

The key idea behind influential instances is to trace model parameters and predictions back to where it all began: the training data. A (parametric) machine learning algorithm is a function that takes training data consisting of features, $X$, and target, $y$, and learns optimal parameters that minimize the loss between true and predicted value. Therefore, a first step for debugging is to see how the model parameters or the predictions would change if we removed instances from the training data in the training process.

In [ ]:
def deletion_diagnostic(X, y, model):
    """
    Function to iteratively train a model on subset of data points to find influential instances.
    :param X: pd.DataFrame with features
    :param y: array with target
    :param model: model pipeline

    :return: list of class probability differences
    """
    # reset indices to simplify loop
    X = X.reset_index(drop=True).copy(deep=True)
    y = y.reset_index(drop=True).copy(deep=True)
    # use original model to make predictions - keep good credit (0)
    y_pred = model.predict_proba(X)[:, [0]]
    # initialize list to store difference between output probabilities
    influence_diff = []
    # loop through dataframe and iteratively drop one data point
    for i in tqdm(range(0, len(X))):
        subset_X = X.drop(index=i)
        subset_y = y.drop(index=i)
        # fit model on subset
        m = model.fit(subset_X, subset_y)
        # get prediction on subset
        subset_y_pred = m.predict_proba(subset_X)[:, [0]]
        # fill current idx value with nan
        subset_y_pred = np.insert(subset_y_pred, i, np.nan)
        # take difference between original and subset value
        influence_diff.append(y_pred.reshape(-1) - subset_y_pred.reshape(-1))
    return np.array(influence_diff)


diff = deletion_diagnostic(X_train, y_train, clf)

In [ ]:
# rows correspond to iteration of training (which in turns correspond to running the learning algorithm on all the dataset minus one example), columns are the probability differences for a given data point
influence_df = pd.DataFrame(np.asmatrix(diff))

# get mean influence value per data point, absolute value first as difference can be pos and neg - avoid cancelling out
influence_vals = np.abs(influence_df).mean(
    skipna=True, axis=0
)  # lf todo, do you have a reference for this procedure? or .... why don't we look at the validation error instead?

# sort from highest diff to lowest
sorted_influential = influence_vals.sort_values(ascending=False)

# look at most influential instances
print(sorted_influential[:5])

The influential instances are the first ones that should be checked for errors, because each error in them strongly influences the model predictions. For example, we could try re-training the model without those instances and check the performance. For more suggestions on how to interpret and work with influential instances have a look at [Koh and Liang](https://arxiv.org/pdf/1703.04730.pdf).

## Thank you for participating!